


# *** FINAL PROJECT - Application of methods for data analysis***

###   *part 1- data collection*

**Goal:** collect emails of five different senders.  
**Plan:** we will use Gmail API in order to connect to our gmail in take emails from there.  

first let's import the relevant packages.  
Brief summary about them:  
email package - this package is for managing email messages, we will use the main class from there called email.message in order to create messages objects and get the relevant information from them.  
imaplib package - implements a client for communicating with Internet Message Access Protocol, we will use it in order to connect to gmail imap.  
re package - regular expression package, we will use it in order to search\remove patterns in the data.

In [2]:
import email
import imaplib
import re

connect to imap.gmail api using IMAP package, and login by user and passwrd (pwd)

In [3]:
user="shavitcherni91"   #to delete later
pwd="tamirshavit"       #to delete later

mail = imaplib.IMAP4_SSL("imap.gmail.com")
mail.login(user,pwd)

('OK', [b'shavitcherni91@gmail.com authenticated (Success)'])

now, after we are logged in, we can send queries to gmail api by the variable mail we created,  
first we will choose folder "INBOX", because the relevant emails will be there.

In [4]:
mail.select("INBOX")

('OK', [b'2'])

this command will take all the emails in INBOX folder and saves their ids in data.  
result will hold the status for our request. (ok,failed)  

about the command: 'search' is the function we want to do, None indicate the parameters for the function, and the third argument is a filter, "ALL" means take everything. (equivalent to no filter)

In [5]:
result, data= mail.uid('search',None, "ALL")
data

[b'3 4']

data now holds a list with one string member that include all the ids.  
let's split it to list that each member is id:

In [6]:
inbox_item_list = data[0].split() 
inbox_item_list

[b'3', b'4']

from this step we dont use api anymore, we have all the data we want and we will take the relevant information from it.  
first, we want to collect the emails of just five senders:

In [7]:
emailAddresses_list = ["shavit91@gmail.com","shavitcherni91@gmail.com","cherniho@post.bgu.ac.il","four@gmail.com","five@gmail.com"]email_message

let's investigate the FROM field as we can see its include the email assress of the sender, but not only... it also include more characters, so we have to extract only the address.  
we will do it inside the loop.

In [21]:
result2, email_data = mail.uid('fetch', inbox_item_list[0], '(RFC822)')
raw_email = email_data[0][1].decode("ISO-8859-1")  
email_message = email.message_from_string(raw_email) 
print("subject: " + email_message['SUBJECT'])
print("from: " + email_message['FROM'])

subject: try
from: =?UTF-8?B?16nXkdeZ15gg16bXqNeg15nXl9eV15E=?= <shavitcherni91@gmail.com>


this snippet include the main functionality, we fetch each email id in order to get the email data. each data is coded in byte, so we have to decode it to ISO-8859-1 in order to get string. after we have the decoded data, we create message by using message_from_string function in email package. the field "FROM" in email_message as we mentioned include the email address but with some more details like name or other signs we dont need so we are using re package in order to get *only* the email address.  

we save the data in a list container, each member is also a list that describe a sender, and each sender is also a list with 2 members, the first one is the subject and the second is the payload data.

In [309]:
lst=[]
for item in inbox_item_list:
    result2, email_data = mail.uid('fetch', item, '(RFC822)')
    raw_email = email_data[0][1].decode("ISO-8859-1")  
    email_message = email.message_from_string(raw_email) 
    m = re.search('[^\s@<>]+@[^\s@<>]+\.[^\s@<>]+',email_message['FROM'])
    lst_member=[]
    if m.group(0) in emailAddresses_list:
        lst_data=[]
        lst_data.append(email_message['SUBJECT'])
        for part in email_message.walk():
            content_type = part.get_content_type()
            if "plain" in content_type:
                lst_data.append(part.get_payload())
            else:
                continue
        lst_member.append(lst_data)
    lst.append(lst_member)

let's see our output list:

In [321]:
lst

[[['try', 'shavit try\r\n']],
 [['hello world', 'another try, another hello world\r\n']]]

as we can see, there are signs like \r\n that are not language meaningful, so we will remove them (using re).  
for now, we decided not to remove numbers, punctuations and other signs.

In [330]:
for i in range(0,len(lst)): #each member
    for j in range(0,len(lst[i])): #each message
        for t in range(0,2):
            lst[i][j][t] = re.sub(r'[^ \w\.]', '', lst[i][j][t])
lst

[[['try', 'shavit try']], [['hello world', 'another try another hello world']]]



*relevant information we used from the internet:*

using IMAP with gmail tutorial: https://yuji.wordpress.com/2011/06/22/python-imaplib-imap-example-with-gmail/  
phyton docs: https://docs.python.org 
building regular expressions: https://regexr.com/  
and lectures material.